In [41]:
import pandas as pd
import tensorflow as tf
import joblib

In [2]:
df = pd.read_csv('./weatherAUS.csv')
# 호주 비 올지 안올지 예측
# https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package

In [3]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


# 데이터 전처리

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [5]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [6]:
df = df[['Date','MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine','RainTomorrow']]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          145460 non-null  object 
 1   MinTemp       143975 non-null  float64
 2   MaxTemp       144199 non-null  float64
 3   Rainfall      142199 non-null  float64
 4   Evaporation   82670 non-null   float64
 5   Sunshine      75625 non-null   float64
 6   RainTomorrow  142193 non-null  object 
dtypes: float64(5), object(2)
memory usage: 7.8+ MB


In [8]:
df = df.dropna()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70115 entries, 6049 to 142302
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          70115 non-null  object 
 1   MinTemp       70115 non-null  float64
 2   MaxTemp       70115 non-null  float64
 3   Rainfall      70115 non-null  float64
 4   Evaporation   70115 non-null  float64
 5   Sunshine      70115 non-null  float64
 6   RainTomorrow  70115 non-null  object 
dtypes: float64(5), object(2)
memory usage: 4.3+ MB


In [10]:
df['RainTomorrow'].unique()

array(['No', 'Yes'], dtype=object)

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()
result = le.fit_transform(df['RainTomorrow'])

In [13]:
df['y_label'] = result

In [14]:
df

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,RainTomorrow,y_label
6049,2009-01-01,17.9,35.2,0.0,12.0,12.3,No,0
6050,2009-01-02,18.4,28.9,0.0,14.8,13.0,No,0
6051,2009-01-03,15.5,34.1,0.0,12.6,13.3,No,0
6052,2009-01-04,19.4,37.6,0.0,10.8,10.6,No,0
6053,2009-01-05,21.9,38.4,0.0,11.4,12.2,No,0
...,...,...,...,...,...,...,...,...
142298,2017-06-20,19.3,33.4,0.0,6.0,11.0,No,0
142299,2017-06-21,21.2,32.6,0.0,7.6,8.6,No,0
142300,2017-06-22,20.7,32.8,0.0,5.6,11.0,No,0
142301,2017-06-23,19.5,31.8,0.0,6.2,10.6,No,0


In [15]:
df['Date'] = pd.to_datetime(df['Date'])

In [16]:
df

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,RainTomorrow,y_label
6049,2009-01-01,17.9,35.2,0.0,12.0,12.3,No,0
6050,2009-01-02,18.4,28.9,0.0,14.8,13.0,No,0
6051,2009-01-03,15.5,34.1,0.0,12.6,13.3,No,0
6052,2009-01-04,19.4,37.6,0.0,10.8,10.6,No,0
6053,2009-01-05,21.9,38.4,0.0,11.4,12.2,No,0
...,...,...,...,...,...,...,...,...
142298,2017-06-20,19.3,33.4,0.0,6.0,11.0,No,0
142299,2017-06-21,21.2,32.6,0.0,7.6,8.6,No,0
142300,2017-06-22,20.7,32.8,0.0,5.6,11.0,No,0
142301,2017-06-23,19.5,31.8,0.0,6.2,10.6,No,0


In [17]:
df.sort_values(by='Date',inplace=True)

In [18]:
df

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,RainTomorrow,y_label
45587,2007-11-01,8.0,24.3,0.0,3.4,6.3,Yes,1
45588,2007-11-02,14.0,26.9,3.6,4.4,9.7,Yes,1
45589,2007-11-03,13.7,23.4,3.6,5.8,3.3,Yes,1
45590,2007-11-04,13.3,15.5,39.8,7.2,9.1,Yes,1
45591,2007-11-05,7.6,16.1,2.8,5.6,10.6,No,0
...,...,...,...,...,...,...,...,...
120637,2017-06-25,4.9,16.8,0.0,1.6,7.9,No,0
33519,2017-06-25,7.6,19.3,0.0,3.4,9.4,No,0
87199,2017-06-25,11.0,24.2,0.0,2.2,9.8,No,0
70392,2017-06-25,8.6,14.3,0.0,2.8,3.8,Yes,1


In [19]:
df.set_index('Date', inplace=True, drop=True)

In [20]:
df.head(10)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,RainTomorrow,y_label
Date,,,,,,,
2007-11-01,8.0,24.3,0.0,3.4,6.3,Yes,1
2007-11-02,14.0,26.9,3.6,4.4,9.7,Yes,1
2007-11-03,13.7,23.4,3.6,5.8,3.3,Yes,1
2007-11-04,13.3,15.5,39.8,7.2,9.1,Yes,1
2007-11-05,7.6,16.1,2.8,5.6,10.6,No,0
2007-11-06,6.2,16.9,0.0,5.8,8.2,No,0
2007-11-07,6.1,18.2,0.2,4.2,8.4,No,0
2007-11-08,8.3,17.0,0.0,5.6,4.6,No,0
2007-11-09,8.8,19.5,0.0,4.0,4.1,Yes,1


In [21]:
# shift를 활용해 다음날 종가 예측 데이터 만들기
df['yesterday_Rainfall'] = df['Rainfall'].shift(periods=1)
df['yesterday_rain'] = df['y_label'].shift(periods=1)

In [22]:
df = df.dropna()

In [23]:
df

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,RainTomorrow,y_label,yesterday_Rainfall,yesterday_rain
Date,,,,,,,,,
2007-11-02,14.0,26.9,3.6,4.4,9.7,Yes,1,0.0,1.0
2007-11-03,13.7,23.4,3.6,5.8,3.3,Yes,1,3.6,1.0
2007-11-04,13.3,15.5,39.8,7.2,9.1,Yes,1,3.6,1.0
2007-11-05,7.6,16.1,2.8,5.6,10.6,No,0,39.8,1.0
2007-11-06,6.2,16.9,0.0,5.8,8.2,No,0,2.8,0.0
...,...,...,...,...,...,...,...,...,...
2017-06-25,4.9,16.8,0.0,1.6,7.9,No,0,0.0,0.0
2017-06-25,7.6,19.3,0.0,3.4,9.4,No,0,0.0,0.0
2017-06-25,11.0,24.2,0.0,2.2,9.8,No,0,0.0,0.0


In [24]:
df['yesterday_rain'] = df['yesterday_rain'].astype('int')

/var/folders/3p/yl9c8k2n4776s934sb09_4dm0000gn/T/ipykernel_1890/1999869702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['yesterday_rain'] = df['yesterday_rain'].astype('int')


In [25]:
df = df.drop(columns='RainTomorrow')

In [26]:
split_time='2015-02-24'

In [27]:
x_train = df[:split_time]
x_valid = df[split_time:]

In [28]:
x_train.shape, x_valid.shape

((56360, 8), (13777, 8))

In [29]:
x = df.drop(columns='y_label')
y = df['y_label'].values

In [30]:
y

array([1, 1, 1, ..., 0, 1, 0])

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y , test_size=0.2, random_state=41)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((56091, 7), (14023, 7), (56091,), (14023,))

In [32]:
x_train

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,yesterday_Rainfall,yesterday_rain
Date,,,,,,,
2011-09-01,2.1,19.6,0.0,2.2,10.1,0.0,0
2011-03-02,11.1,19.8,0.4,3.6,10.3,0.0,0
2016-01-02,14.0,18.2,0.0,5.6,2.0,0.0,0
2012-06-07,2.7,14.6,0.0,2.0,7.6,0.0,0
2017-01-23,12.7,25.0,0.0,9.2,12.7,0.0,0
...,...,...,...,...,...,...,...
2014-06-08,5.8,14.9,1.8,0.6,3.7,0.0,0
2014-10-12,10.0,33.9,0.0,7.6,12.0,0.0,1
2015-10-22,15.0,20.7,0.0,4.0,3.8,0.8,0


# Linear Regression

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

In [34]:
model = LinearRegression()

In [35]:
history = model.fit(x_train,y_train)

In [36]:
y_pred = model.predict(x_test)

In [37]:
y_pred

array([0.01777362, 0.35058729, 0.44938552, ..., 0.32636587, 0.06175019,
       0.11477017])

In [39]:
y_test[3]

0

In [56]:
num = 0
for i in range(len(y_pred)) :
    if round(y_pred[i]) == y_test[i] :
        num += 1
print("accuarcy : " + str(num/len(y_pred)))

accuarcy : 0.808243599800328


In [57]:
joblib.dump(model, './ml_model_linear.h5')

['./ml_model_linear.h5']

In [58]:
model_rf = RandomForestClassifier()

In [59]:
model_rf.fit(x_train,y_train)

RandomForestClassifier()

In [60]:
y_predict_rf = model_rf.predict(x_test)

In [61]:
num = 0
for i in range(len(y_predict_rf)) :
    if y_predict_rf[i] == y_test[i] :
        num += 1
print("accuarcy : " + str(num/len(y_predict_rf)))

accuarcy : 0.8188690009270484


In [62]:
result_df = pd.DataFrame(y_test)

In [63]:
result_df['linear_predict'] = y_pred
result_df['rf_predict'] = y_predict_rf

In [65]:
result_df.to_csv('./weatheraus_predict.csv')